In [1]:
# !pip install tensorflow

In [2]:
import numpy as np
import mnist

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical

In [3]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [4]:
# Нормализация изображений
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

In [5]:
# в вектор длиной 784
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [6]:
# создание модели
model = Sequential([
  Dense(64, activation='elu', input_shape=(784,)),  
  Dense(64, activation='elu'),
  Dense(10, activation='sigmoid'),
])

In [7]:
# компиляция модели
model.compile(
  optimizer='nadam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [8]:
# обучение модели
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=25,
  batch_size=30,
)

Epoch 1/25
2000/2000 [==============================] - 4s 1ms/step - loss: 0.3129 - accuracy: 0.9053
Epoch 2/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1526 - accuracy: 0.9542
Epoch 3/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1143 - accuracy: 0.9643
Epoch 4/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0923 - accuracy: 0.9710
Epoch 5/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0808 - accuracy: 0.9742
Epoch 6/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0724 - accuracy: 0.9774
Epoch 7/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0653 - accuracy: 0.9790
Epoch 8/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0567 - accuracy: 0.9813
Epoch 9/25
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0544 - accuracy: 0.9823
Epoch 10/25
2000/2000 [==============================] - 3s 1ms/step - loss: 0.048

In [9]:
# оценка модели
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

313/313 [==============================] - 0s 1ms/step - loss: 0.1133 - accuracy: 0.9727


[0.11330216377973557, 0.9726999998092651]

In [10]:
# сохранение модели
model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

In [20]:
# прогноз на первых 50 изображениях
predictions = model.predict(test_images[:25])

1/1 [==============================] - 0s 19ms/step


In [21]:
# вывод предсказаний
print(np.argmax(predictions, axis=1))

[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4]


In [22]:
# правильные ответы
print(test_labels[:25])

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4]


In [23]:
predictions = model.predict(test_images[:25])
print(np.argmax(predictions, axis=1))

print(test_labels[:25])

1/1 [==============================] - 0s 18ms/step
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4]


======================================================================================================================== 

Что я, собственно, делал, помимо (Ctrl+C)+(Ctrl+V):

**1. ОПТИМИЗАТОРЫ**

$А. Adam:$ 

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3522 - accuracy: 0.8943

Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1019 - accuracy: 0.9676


$B. RMSprop:$ 

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3635 - accuracy: 0.8885
 
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1130 - accuracy: 0.9664

$C. Nadam:$

Epoch 1/5
1875/1875 [==============================] - 12s 5ms/step - loss: 0.3403 - accuracy: 0.8981

Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0973 - accuracy: 0.9701

$D. SGD$

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.6682 - accuracy: 0.8200

Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2135 - accuracy: 0.9378

$E.Ftrl$

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3024 - accuracy: 0.1121

Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3019 - accuracy: 0.1124


**ВЫВОДЫ:**

Хоть и провел анализ оптимизаторов по офф документации $Keras$, я еще недостаточно зрел, чтобы полностью понять их механику.

Не уверен, что в случае нейросетей исследователи прям сражаются за (0.9701 - 0.9676) точности, но в конкретном случае $Nadam$ выигрышнее смотрится. 

$Ftrl$ вообще не туда. Видимо шаг стоит увеличить, раз он не выбрался из локального минимума, но я не могу с точностью утверждать.
Более разреженных данных требует, видимо. Зато он гениально нашел здесь единицу: [7 2 1 0 4] => [1 1 1 1 1].

Ладно, идем дальше.

**2. АКТИВАЦИЯ**

Далее я решил играться с функциями активации. Здесь нейросетка превратилась в еще более черный ящик. Подбираем комбинации опытным путем.

Полностью relu: 0.0912

Полностью softmax: 0.7132

Полностью sigmoid: 0.9673

sigmoid-reLU-softmax: 0.9694

elu-relu-sigmoid: 0.9742

elu-elu-sigmoid: 0.9748


**ВЫВОДЫ:**

Я только понял, что на выходном слое должна быть softmax или sigmoid, раз мы просчитываем вероятность. Первые два слоя - пальцем в небо.

**3. БАТЧИ**

Размер батча в разумных пределах существенно на метрику не влияет. 

Да, батчей должно быть достаточно для качества обучения.

batch_size=300  ===>  $0.9574$

batch_size=20   ===>  $0.9744$

Ну и не слишком много, чтобы не тратить времени впустую.

batch_size=5   ===> 81s - accuracy: 0.9700

batch_size=30  ===> 16s - accuracy: 0.9751

**4. ЭПОХИ**

epochs=5, accuracy: 0.9751

epochs=7, accuracy: 0.9785

epochs=10, accuracy: 0.9845

epochs=20, accuracy: 0.9907

Рост замедляется до прямой около 0.99, график точности от эпох легко представить.

На этом я закончу. В принципе, для повышения точности увеличить количество эпох было бы достаточно.